In [1]:
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import joblib
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard
%matplotlib inline
sn.set_context("talk")
from sklearn.preprocessing import OneHotEncoder

2024-03-11 18:27:34.339571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:27:34.339593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:27:34.340260: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 18:27:34.343771: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 18:27:34.849895: W tensorflow/compiler/tf2

In [2]:
OneHotEncodder = joblib.load("../data/HMD/hmd-ma_clf_enc.joblib")

In [3]:
len(OneHotEncodder.categories_[0])

7

In [4]:
with tf.device("cpu"):
    train = pd.read_csv("../data/HMD/train_larg.csv")
    test = pd.read_csv("../data/HMD/test_larg.csv")
    sequences = pd.concat([train,test], axis = 0).Sequence.apply(lambda x: " ".join(x))

    X_train = tf.convert_to_tensor(np.array([" ".join(x) for x in train.Sequence.to_list()]))
    X_test = tf.convert_to_tensor(np.array([" ".join(x) for x in test.Sequence.to_list()]))
    y_train = tf.convert_to_tensor(OneHotEncodder.transform([[x] for x in train.Class.to_list()]).toarray())
    y_test = tf.convert_to_tensor(OneHotEncodder.transform([[x] for x in test.Class.to_list()]).toarray())

2024-03-11 18:27:35.421785: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 18:27:35.441603: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 18:27:35.441740: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
max([len(x) for x in train.Sequence.to_list()])

10746

In [6]:
log_dir_cnn  = "../data/HMD/log/gridsearch/cnn-hmd-paramns"
log_dir_same = "../data/HMD/log/gridsearch/cnn-same-hmd.paramns"
log_dir_gru  = "../data/HMD/log/gridsearch/gru-hmd-paramns"

Def models

In [7]:
FILTERS = hp.HParam('num_units', hp.Discrete([32, 64, 128, 512]))
EMB_DIM = hp.HParam('output_dim', hp.Discrete([40, 50, 60, 70]))
METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer(log_dir_cnn).as_default():
  hp.hparams_config(
    hparams=[FILTERS,EMB_DIM],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])


In [8]:
def CNN(hparams,sequences):
    model = tf.keras.models.Sequential()        
    model.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = 1700))
    model.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = hparams[EMB_DIM], input_length = 1700, mask_zero = True))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS],kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS]/2,kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS]/4,kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS]/8,kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.GlobalAveragePooling1D())
    model.add(tf.keras.layers.Dense(7, activation = 'softmax'))
    model.layers[0].adapt(sequences)
    model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),loss = tf.keras.losses.CategoricalCrossentropy(),metrics = [tf.metrics.CategoricalAccuracy()])
    model.fit(X_train, y_train, epochs=20)
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy


In [9]:
def GRU_(hparams,sequences):
    model = tf.keras.models.Sequential()        
    model.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = 1700))
    model.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = hparams[EMB_DIM], input_length = 1700, mask_zero = True))
    model.add(tf.keras.layers.GRU(hparams[FILTERS],return_sequences=True))
    model.add(tf.keras.layers.GRU(hparams[FILTERS],return_sequences=True))
    model.add(tf.keras.layers.GlobalAveragePooling1D())
    model.add(tf.keras.layers.Dense(7, activation = 'softmax'))
    model.layers[0].adapt(sequences)
    model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),loss = tf.keras.losses.CategoricalCrossentropy(),metrics = [tf.metrics.CategoricalAccuracy()])
    model.fit(X_train, y_train, epochs=20)
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [10]:
def CNN_SAME(hparams,sequences):
    model = tf.keras.models.Sequential()        
    model.add(tf.keras.layers.TextVectorization(max_tokens = 21 + 2, output_mode = "int", standardize = None, output_sequence_length = 1700))
    model.add(tf.keras.layers.Embedding(input_dim = 21 + 2, output_dim = hparams[EMB_DIM], input_length = 1700, mask_zero = True))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS],kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS],kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS],kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.Conv1D(filters = hparams[FILTERS],kernel_size = 5, activation = "relu"))
    model.add(tf.keras.layers.GlobalAveragePooling1D())
    model.add(tf.keras.layers.Dense(7, activation = 'softmax'))
    model.layers[0].adapt(sequences)
    model.compile(optimizer = tf.optimizers.Adam(learning_rate = 0.001),loss = tf.keras.losses.CategoricalCrossentropy(),metrics = [tf.metrics.CategoricalAccuracy()])
    model.fit(X_train, y_train, epochs=20)
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [11]:
def run_cnn(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = CNN(hparams,sequences)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
  
def run_cnn_same(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = CNN_SAME(hparams,sequences)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    
def run_gru(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = GRU_(hparams,sequences)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [12]:

session_num = 0
for filters in FILTERS.domain.values:
    for output_dim in EMB_DIM.domain.values:
        hparams = {
            FILTERS: filters,
            EMB_DIM: output_dim,
            }
        
        try:
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})        
            run_cnn(log_dir_cnn + "/" + run_name, hparams) #Fit function
        except:
            pass
        
        try:
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})   
            run_cnn_same(log_dir_same + "/" + run_name, hparams) #Fit function
        except:
            pass

        try:
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})   
            run_gru(log_dir_gru + "/" + run_name, hparams) #Fit functionru
        except:
            pass


        session_num +=1

--- Starting trial: run-0
{'num_units': 32, 'output_dim': 40}
Epoch 1/20


2024-03-11 18:27:38.870796: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 18:27:40.013791: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-11 18:27:40.433641: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 18:27:40.697091: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f807000c1d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 18:27:40.697108: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-03-11 18:27:40.701193: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710192460.775499  749272 device_compiler.

394/394 [==============================] - 6s 6ms/step - loss: 1.5933 - categorical_accuracy: 0.3866
Epoch 2/20
394/394 [==============================] - 2s 4ms/step - loss: 1.0881 - categorical_accuracy: 0.6299
Epoch 3/20
394/394 [==============================] - 2s 4ms/step - loss: 1.0036 - categorical_accuracy: 0.6514
Epoch 4/20
394/394 [==============================] - 2s 4ms/step - loss: 0.9674 - categorical_accuracy: 0.6606
Epoch 5/20
394/394 [==============================] - 2s 4ms/step - loss: 0.9487 - categorical_accuracy: 0.6635
Epoch 6/20
394/394 [==============================] - 2s 4ms/step - loss: 0.9289 - categorical_accuracy: 0.6692
Epoch 7/20
394/394 [==============================] - 2s 4ms/step - loss: 0.9073 - categorical_accuracy: 0.6765
Epoch 8/20
394/394 [==============================] - 2s 4ms/step - loss: 0.8794 - categorical_accuracy: 0.6861
Epoch 9/20
394/394 [==============================] - 2s 4ms/step - loss: 0.8327 - categorical_accuracy: 0.7038
Epo

2024-03-11 18:29:09.441363: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT8
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_35/output/_22'


394/394 [==============================] - 21s 38ms/step - loss: 1.2884 - categorical_accuracy: 0.5160
Epoch 2/20
394/394 [==============================] - 15s 37ms/step - loss: 0.9131 - categorical_accuracy: 0.6630
Epoch 3/20
394/394 [==============================] - 15s 37ms/step - loss: 0.8437 - categorical_accuracy: 0.6872
Epoch 4/20
394/394 [==============================] - 15s 38ms/step - loss: 0.7838 - categorical_accuracy: 0.7116
Epoch 5/20
394/394 [==============================] - 15s 38ms/step - loss: 0.7248 - categorical_accuracy: 0.7327
Epoch 6/20
394/394 [==============================] - 15s 38ms/step - loss: 0.6810 - categorical_accuracy: 0.7525
Epoch 7/20
394/394 [==============================] - 15s 38ms/step - loss: 0.6342 - categorical_accuracy: 0.7855
Epoch 8/20
394/394 [==============================] - 15s 38ms/step - loss: 0.5898 - categorical_accuracy: 0.8025
Epoch 9/20
394/394 [==============================] - 15s 38ms/step - loss: 0.5514 - categorical_ac